# Data Loading

In [60]:
import pandas as pd
import numpy as np

In [61]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

age  income student credit_rating class_buys_computer
0         youth    high      no          fair                  no
1         youth    high      no     excellent                  no
2   middle_aged    high      no          fair                 yes
3        senior  medium      no          fair                 yes
4        senior     low     yes          fair                 yes
5        senior     low     yes     excellent                  no
6   middle_aged     low     yes     excellent                 yes
7         youth  medium      no          fair                  no
8         youth     low     yes          fair                 yes
9        senior  medium     yes          fair                 yes
10        youth  medium     yes     excellent                 yes
11  middle_aged  medium      no     excellent                 yes
12  middle_aged    high     yes          fair                 yes
13       senior  medium      no     excellent                  no

## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [62]:
def get_gini(df, label):
    label_freq = df[label].value_counts() / len(df)
    gini = 1 - (label_freq**2).sum()
    return gini

In [63]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [64]:
from itertools import combinations

def get_binary_split(df, attribute):

    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트

    result = [list(item) for item in combinations(uniques,1)] + [list(item) for item in combinations(uniques, 2)]
    return result

In [65]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [68]:
def gini_index(yes_count, no_count):
    total = yes_count + no_count
    if total == 0:
        return 0
    return 1 - ((yes_count / total) ** 2) - ((no_count / total) ** 2)

def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)

    for cls in binary_split:
        left_indices = [idx for idx, val in enumerate(df[attribute]) if val in cls]
        right_indices = [idx for idx, val in enumerate(df[attribute]) if val not in cls]

        left_yes = sum(df[label].iloc[idx] == 'yes' for idx in left_indices)
        right_yes = sum(df[label].iloc[idx] == 'yes' for idx in right_indices)

        left_no = len(left_indices) - left_yes
        right_no = len(right_indices) - right_yes

        gini_left = gini_index(left_yes, left_no)
        gini_right = gini_index(right_yes, right_no)

        gini = (len(left_indices) / n) * gini_left + (len(right_indices) / n) * gini_right
        result[tuple(cls)] = gini

    return result


In [69]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [70]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [71]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [72]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
  gini_indexes = get_attribute_gini_index(pd_data, feature, label)
  min_gini_index = min(gini_indexes.values())
  print(f'Min Gini Index of: {feature}, : {min_gini_index}')


Min Gini Index of: age, : 0.35714285714285715
Min Gini Index of: income, : 0.4428571428571429
Min Gini Index of: student, : 0.3673469387755103
Min Gini Index of: credit_rating, : 0.42857142857142855


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [73]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [74]:
from math import log2

def getEntropy(df, feature):
    values, counts = np.unique(df[feature], return_counts=True)
    total = float(sum(counts))
    entropy = 0.0

    for count in counts:
        prob = count / total
        entropy -= prob * log2(prob)

    return entropy


In [75]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [76]:
def getGainA(df, target):

    result = {}

    base_entropy = getEntropy(df, target)

    for feature in df.columns:
        if feature == target:
            continue

        unique_values = df[feature].unique()
        feature_entropy = 0.0
        for value in unique_values:
            subset = df[df[feature] == value]
            value_prob = len(subset) / float(len(df))
            feature_entropy += value_prob * getEntropy(subset, target)

        result[feature] = base_entropy - feature_entropy

    return result


In [77]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}